In [ ]:
wd<-"/Seurat/reg.CC/WT_replace_v2/DEGs/SCT/res02_220622/"
if(!dir.exists(wd))
    dir.create(wd)
#dir.create(wd)
setwd(wd)
getwd()

In [ ]:
library(Seurat)
library(ggplot2)
library(patchwork)
library(dplyr)
library(hdf5r)
library(png)
library(ggalluvial)
#library(tidyverse)
library(clustree)
library(RCTD)
library(Matrix)
#library(SPOTlight)
library(future)
library(grid)
library(RColorBrewer)
library(pheatmap)
library(circlize)
library(igraph)
library(ComplexHeatmap)
library(stringr)
library(ComplexHeatmap)
library(cowplot)
library(clusterProfiler)
options(connectionObserver = NULL)
library(biomaRt)
#library(scClustViz)


In [ ]:
da<-readRDS("/home/jovyan/zxli_SCI/result/Seurat/reg.CC/WT_replace_v2/res02_220310/WT.merge.replace_v2.SCT.regress_CC.nC.mt.ident.pc20.k50.res02.rds")
da

In [ ]:
backgenes<-read.table("/home/jovyan/zxli_SCI/result/Seurat/reg.CC/WT_replace_v2/WT.GOterm.SCT.backgeneids.txt")
backgenes<-as.character(backgenes$x)
head(backgenes)

1. time bulk DEGs vs. sham

In [ ]:
Idents(da)<-da$time
levels(Idents(da))

In [4]:
sd<-"time_vs_sham.marker/"
if(!dir.exists(sd))
    dir.create(sd)

In [44]:

for(a in c(levels(Idents(da))[2:4])){
    mark<-FindMarkers(da,logfc.threshold = 0.5,min.pct = 0.25,only.pos = F,ident.1=a,ident.2="WT_sham")
        
    mark <- filter(mark,p_val_adj<0.01)
    dim(mark)
    mark$group<-a
    table(mark$group)
    write.csv(mark,paste0(sd,a,".vs.sham_Marker.np.fc05.pct025.p001.csv"))
    
}


In [ ]:
f<-dir(sd)
f

In [ ]:
temp<-read.csv(paste0(sd,"WT_24h.vs.sham_Marker.np.fc05.pct025.p001.csv"))
head(temp)
for(i in 2:length(f)){
    temp2<-read.csv(paste0(sd,f[i]))
    temp<-rbind(temp,temp2)
}
dim(temp)

In [ ]:
temp$up_down<-""
temp$up_down[temp$avg_log2FC>0]<-"up"
temp$up_down[temp$avg_log2FC<0]<-"down"
table(temp$up_down)

In [ ]:
temp$time<-temp$group
temp$group<-paste0(temp$time,"-",temp$up_down)
table(temp$group)

In [ ]:
table(temp$group)
write.csv(temp,paste0(sd,"time.bulk.vs.sham.fc05.pct025.pad001.rbind.csv"))

In [ ]:
library(clusterProfiler)
options(connectionObserver = NULL)
#remotes::install_version("RSQLite", version = "2.2.5")
library(org.Mm.eg.db)
library(ggplot2)
#grlabs<-split(gene.nodle.df$gene,gene.nodle.df$module)
grlabs<-split(temp$X,temp$group)
gcSample = lapply(grlabs, function(gr) as.numeric(bitr(gr, fromType = "SYMBOL", toType = "ENTREZID", OrgDb = "org.Mm.eg.db")$ENTREZID))
pvalueCutoff = 0.01
qvalueCutoff = 0.01
xx.mus.go <- compareCluster(gcSample, OrgDb='org.Mm.eg.db', fun='enrichGO',#maxGSSize=150,
                            pvalueCutoff = pvalueCutoff, qvalueCutoff = qvalueCutoff,
                            ont = "BP", readable=T,universe=backgenes
                           )
EnrichResGO_mm = xx.mus.go@compareClusterResult
saveRDS(xx.mus.go,paste0(sd,"WT.time.vs.sham.fc05.pct025.p001.updown_pqval001.background.go.rds"))

In [6]:
xx.mus.go<-readRDS(paste0(sd,"WT.domain.time.vs.sham.fc05.pct025.p001.updown_pqval001.background.go.rds"))

In [ ]:
###ggplot2
df<-xx.mus.go@compareClusterResult
head(df)
dim(df)
unique(df$Cluster)

In [15]:
df1 = df %>% group_by(.,Cluster) %>% filter(Count>=5) %>% top_n(.,-5,p.adjust)  #按照Cluster排序，并取每个cluster的前20个
#df1<-df[df$Description %in% unique(df1$Description),]
df1$Cluster<-factor(df1$Cluster,levels = c('WT_3h-up','WT_24h-up','WT_72h-up','WT_3h-down','WT_24h-down','WT_72h-down'))
df1<-df1[order(df1$Cluster),]
df1$Description = factor(df1$Description,levels = rev(unique(df1$Description)))  #对Description进行按cluster排序，后面展示时同一cluster的go可以放在一起


In [12]:
df_up<-df1[grep("up",df1$Cluster),]

In [ ]:
options(repr.plot.width=6, repr.plot.height=5,font.size=15)
library(ggplot2)
p=ggplot(df_up,aes(Cluster,Description))+geom_point(aes(size=Count,color=p.adjust))+        #基因数目表示SIZE，p.adjust代表颜色深浅
    scale_color_gradientn(colors=brewer.pal(9,"Spectral"),guide = guide_colorbar(reverse = T))+
    theme_bw()+
    theme(axis.title = element_blank(),axis.text = element_text(size = 13),axis.text.x.bottom = element_text(angle = 90))#+facet_grid(ONTOLOGY~., scales = "free")
p
ggsave(paste0(sd,"WT.time.upMarker.fc05.pct025.p001_GO.pqval001.C5.pval.top5.go_220627.png"),width = 6,height = 5,dpi = 400)

In [16]:
df_down<-df1[grep("down",df1$Cluster),]

In [ ]:
options(repr.plot.width=5.5, repr.plot.height=5,font.size=15)
library(ggplot2)
p=ggplot(df_down,aes(Cluster,Description))+geom_point(aes(size=Count,color=p.adjust))+        #基因数目表示SIZE，p.adjust代表颜色深浅
    scale_color_gradientn(colors=brewer.pal(9,"Spectral"),guide = guide_colorbar(reverse = T))+
    theme_bw()+
    theme(axis.title = element_blank(),axis.text = element_text(size = 13),axis.text.x.bottom = element_text(angle = 90))#+facet_grid(ONTOLOGY~., scales = "free")
p
ggsave(paste0(sd,"WT.time.downMarker.fc05.pct025.p001_GO.pqval001.C5.pval.top5.go_220627.png"),width = 5.5,height = 5,dpi = 400)

2. HT DEGs

In [ ]:
Hda<-da[rownames(da),rownames(da@meta.data[da@meta.data$RC=="WT_H",])]
unique(Hda$RC)

In [ ]:
Tda<-da[rownames(da),rownames(da@meta.data[da@meta.data$RC=="WT_T",])]
unique(Tda$RC)

In [ ]:
HT<-c(Hda,Tda)
HT

In [ ]:
names(HT)<-c("H","T")
HT

In [108]:
for(j in c("H","T")){
    temp2<-HT[[j]]
    #name<-names(HT[j])
    for(a in c(levels(Idents(temp2))[2:4])){
        mark<-FindMarkers(temp2,logfc.threshold = 0.5,min.pct = 0.25,only.pos = F,ident.1=a,ident.2="WT_sham")
            
        mark <- filter(mark,p_val_adj<0.01)
        dim(mark)
        mark$group<-paste0(a,"_",j)
        table(mark$group)
        write.csv(mark,paste0(sd,j,"_",a,".timevs.sham_Marker.fc05.pct025.p001.csv"))
    }
}

In [ ]:
f<-dir(sd,pattern = "^[H-T]")
f

In [ ]:
temp<-read.csv(paste0(sd,f[1]))
head(temp)
for(i in 2:length(f)){
    temp2<-read.csv(paste0(sd,f[i]))
    temp<-rbind(temp,temp2)
}
dim(temp)

In [ ]:
temp$up_down<-""
temp$up_down[temp$avg_log2FC>0]<-"up"
temp$up_down[temp$avg_log2FC<0]<-"down"
table(temp$up_down)

In [ ]:
temp$time<-temp$group
temp$group<-paste0(temp$time,"-",temp$up_down)
table(temp$group)

In [ ]:
table(temp$group)
write.csv(temp,paste0(sd,"HT.time.bulk.vs.sham.fc05.pct025.pad001.rbind.csv"))

In [ ]:
library(clusterProfiler)
options(connectionObserver = NULL)
#remotes::install_version("RSQLite", version = "2.2.5")
library(org.Mm.eg.db)
library(ggplot2)
#grlabs<-split(gene.nodle.df$gene,gene.nodle.df$module)
grlabs<-split(temp$X,temp$group)
gcSample = lapply(grlabs, function(gr) as.numeric(bitr(gr, fromType = "SYMBOL", toType = "ENTREZID", OrgDb = "org.Mm.eg.db")$ENTREZID))
pvalueCutoff = 0.01
qvalueCutoff = 0.01
xx.mus.go <- compareCluster(gcSample, OrgDb='org.Mm.eg.db', fun='enrichGO',#maxGSSize=150,
                            pvalueCutoff = pvalueCutoff, qvalueCutoff = qvalueCutoff,
                            ont = "BP", readable=T,universe=backgenes
                           )
EnrichResGO_mm = xx.mus.go@compareClusterResult
saveRDS(xx.mus.go,paste0(sd,"WT.HT.time.vs.sham.fc05.pct025.p001.updown_pqval001.background.go.rds"))

In [26]:
xx.mus.go<-readRDS(paste0(sd,"WT.HT.time.vs.sham.fc05.pct025.p001.updown_pqval001.background.go.rds"))

3. distance

In [ ]:
Hda<-da[rownames(da),rownames(da@meta.data[da@meta.data$distance=="1mm",])]
unique(Hda$distance)

In [ ]:
Tda<-da[rownames(da),rownames(da@meta.data[da@meta.data$distance=="0.5mm",])]
unique(Tda$distance)

In [ ]:
HT<-c(Hda,Tda)
HT

In [ ]:
names(HT)<-c("1mm","0.5mm")
HT

In [148]:
for(j in c("1mm","0.5mm")){
    temp2<-HT[[j]]
    #name<-names(HT[j])
    for(a in c(levels(Idents(temp2))[2:4])){
        mark<-FindMarkers(temp2,logfc.threshold = 0.5,min.pct = 0.25,only.pos = F,ident.1=a,ident.2="WT_sham")
            
        mark <- filter(mark,p_val_adj<0.01)
        dim(mark)
        mark$group<-paste0(a,"_",j)
        table(mark$group)
        write.csv(mark,paste0(sd,j,"_",a,".timevs.sham_Marker.fc05.pct025.p001.csv"))
    }
}

In [ ]:
f<-dir(sd,pattern = "^[0-1]")
f

In [ ]:
temp<-read.csv(paste0(sd,f[1]))
head(temp)
for(i in 2:length(f)){
    temp2<-read.csv(paste0(sd,f[i]))
    temp<-rbind(temp,temp2)
}
dim(temp)

In [ ]:
temp$up_down<-""
temp$up_down[temp$avg_log2FC>0]<-"up"
temp$up_down[temp$avg_log2FC<0]<-"down"
table(temp$up_down)

In [ ]:
temp$time<-temp$group
temp$group<-paste0(temp$time,"-",temp$up_down)
table(temp$group)

In [11]:
write.csv(temp,paste0(sd,"distance.time.bulk.vs.sham.fc05.pct025.pad001.rbind.csv"))

4. domain

In [ ]:
temp<-read.csv("/Seurat/reg.CC/WT_replace_v2/res02_220310/time_domain.marker/WT.domain.timevs.sham.fc05.pct025.p001.np.marker.csv")
temp<-temp[,-1]
temp$time<-paste0(str_split(temp$group,"_",simplify = T)[,1],"_",str_split(temp$group,"_",simplify = T)[,2])
head(temp)

In [ ]:
dim(temp)
temp$up_down<-""
temp$up_down[temp$avg_log2FC>0]<-"up"
temp$up_down[temp$avg_log2FC<0]<-"down"
table(temp$up_down)

In [ ]:
temp$group<-paste0(temp$group,"-",temp$up_down)
table(temp$group)

In [16]:
write.csv(temp,paste0(sd,"domain.time.bulk.vs.sham.fc05.pct025.pad001.rbind.csv"))

In [ ]:
library(clusterProfiler)
options(connectionObserver = NULL)
#remotes::install_version("RSQLite", version = "2.2.5")
library(org.Mm.eg.db)
library(ggplot2)
#grlabs<-split(gene.nodle.df$gene,gene.nodle.df$module)
grlabs<-split(temp$X,temp$group)
gcSample = lapply(grlabs, function(gr) as.numeric(bitr(gr, fromType = "SYMBOL", toType = "ENTREZID", OrgDb = "org.Mm.eg.db")$ENTREZID))
pvalueCutoff = 0.01
qvalueCutoff = 0.01
xx.mus.go <- compareCluster(gcSample, OrgDb='org.Mm.eg.db', fun='enrichGO',#maxGSSize=150,
                            pvalueCutoff = pvalueCutoff, qvalueCutoff = qvalueCutoff,
                            ont = "BP", readable=T,universe=backgenes
                           )
EnrichResGO_mm = xx.mus.go@compareClusterResult
saveRDS(xx.mus.go,paste0(sd,"WT.domain.time.vs.sham.fc05.pct025.p001.updown_pqval001.background.go.rds"))

In [158]:
xx.mus.go<-readRDS(paste0(sd,"WT.domain.time.vs.sham.fc05.pct025.p001.updown_pqval001.background.go.rds"))

In [ ]:
###ggplot2
df<-xx.mus.go@compareClusterResult
head(df)
dim(df)
unique(df$Cluster)

#### show each group individually

In [ ]:
df$time<-paste0(str_split(df$Cluster,"_",simplify = T)[,1],"_",str_split(df$Cluster,"_",simplify = T)[,2])
df$time<-factor(df$time,levels = c("WT_3h","WT_24h","WT_72h"))
df<-df[order(df$time),]
df$up_down<-str_split(df$Cluster,"-",simplify = T)[,2]
df$up_down<-factor(df$up_down,levels = c("up","down"))
df<-df[order(df$up_down),]
head(df)

In [55]:
write.csv(df,"WT.domain.time.vs.sham.fc05.pct025.p001.updown_pqval001.background.go.adjust.dataframe.csv")

In [187]:
df2<-df[grep("DH",df$Cluster),]
df1 = df1 %>% group_by(.,Cluster) %>% filter(Count>=5) %>% top_n(.,-5,p.adjust)  #按照Cluster排序，并取每个cluster的前20个
df1<-df2[df2$Description %in% unique(df1$Description),]
df1$Cluster<-factor(df1$Cluster,levels = c('WT_3h_DH-up','WT_3h_MG-up','WT_3h_VH-up','WT_3h_WM-up',
                                           'WT_24h_DH-up','WT_24h_MG-up','WT_24h_VH-up','WT_24h_WM-up',
                                           'WT_72h_DH-up','WT_72h_MG-up','WT_72h_VH-up','WT_72h_WM-up',
                                           'WT_3h_DH-down','WT_3h_MG-down','WT_3h_VH-down','WT_3h_WM-down',
                                           'WT_24h_DH-down','WT_24h_MG-down','WT_24h_VH-down','WT_24h_WM-down',
                                           'WT_72h_DH-down','WT_72h_MG-down','WT_72h_VH-down','WT_72h_WM-down'
                                          ))
df1<-df1[order(df1$time),]
df1<-df1[order(df1$up_down),]
#if(unique(df1$up_down)=="up"){
#    df1$Description = factor(df1$Description,
#                         levels = c(unique(df1$Description)))  #对Description进
#}else{
#    df1$Description = factor(df1$Description,
#                         levels = c(unique(df1$Description)))
#}
df1$Description = factor(df1$Description,
                         levels = unique(c(unique(df1[df1$up_down=="up",]$Description),rev(unique(df1[df1$up_down=="down",]$Description)))))  #对Description进

In [ ]:
#df_up<-df1[grep("up",df1$Cluster),]
options(repr.plot.width=6, repr.plot.height=4,font.size=15)
library(ggplot2)
p=ggplot(df1,aes(time,Description))+geom_point(aes(size=Count,color=p.adjust))+        #基因数目表示SIZE，p.adjust代表颜色深浅
    scale_color_gradientn(colors=brewer.pal(9,"Spectral"),guide = guide_colorbar(reverse = T))+
    theme_bw()+facet_wrap(~up_down,nrow=2,scales = "free")+
    theme(axis.title = element_blank(),axis.text = element_text(size = 13),axis.text.x.bottom = element_text(angle = 90))#+facet_grid(ONTOLOGY~., scales = "free")
p
ggsave(paste0(sd,"WT.DH.time.Marker.fc05.pct025.p001_GO.pqval001.C5.pval.top5.go_updown.png"),width = 6,height = 8.5,dpi = 400)

In [173]:
df2<-df[grep("MG",df$Cluster),]
df1 = df1 %>% group_by(.,Cluster) %>% filter(Count>=5) %>% top_n(.,-5,p.adjust)  #按照Cluster排序，并取每个cluster的前20个
df1<-df2[df2$Description %in% unique(df1$Description),]
#df1$Cluster<-factor(df1$Cluster,levels = c('WT_3h_DH-up','WT_3h_MG-up','WT_3h_VH-up','WT_3h_WM-up',
#                                           'WT_24h_DH-up','WT_24h_MG-up','WT_24h_VH-up','WT_24h_WM-up',
#                                           'WT_72h_DH-up','WT_72h_MG-up','WT_72h_VH-up','WT_72h_WM-up',
#                                           'WT_3h_DH-down','WT_3h_MG-down','WT_3h_VH-down','WT_3h_WM-down',
#                                           'WT_24h_DH-down','WT_24h_MG-down','WT_24h_VH-down','WT_24h_WM-down',
#                                           'WT_72h_DH-down','WT_72h_MG-down','WT_72h_VH-down','WT_72h_WM-down'
#                                          ))
df1<-df1[order(df1$time),]
df1<-df1[order(df1$up_down),]
df1$Description = factor(df1$Description,
                         levels = unique(c(unique(df1[df1$up_down=="up",]$Description),rev(unique(df1[df1$up_down=="down",]$Description)))))  #对Description进

In [ ]:
options(repr.plot.width=6.2, repr.plot.height=8.5,font.size=15)
library(ggplot2)
p=ggplot(df1,aes(time,Description))+geom_point(aes(size=Count,color=p.adjust))+        #基因数目表示SIZE，p.adjust代表颜色深浅
    scale_color_gradientn(colors=brewer.pal(9,"Spectral"),guide = guide_colorbar(reverse = T))+
    theme_bw()+facet_wrap(~up_down,nrow=2,scales = "free")+
    theme(axis.title = element_blank(),axis.text = element_text(size = 13),axis.text.x.bottom = element_text(angle = 90))#+facet_grid(ONTOLOGY~., scales = "free")
p
#ggsave(paste0(sd,"WT.MG.time.Marker.fc05.pct025.p001_GO.pqval001.C5.pval.top5.go_updown.png"),width = 6.2,height = 8.5,dpi = 400)

In [176]:
df2<-df[grep("VH",df$Cluster),]
df1 = df1 %>% group_by(.,Cluster) %>% filter(Count>=5) %>% top_n(.,-5,p.adjust)  #按照Cluster排序，并取每个cluster的前20个
df1<-df2[df2$Description %in% unique(df1$Description),]
#df1$Cluster<-factor(df1$Cluster,levels = c('WT_3h_DH-up','WT_3h_MG-up','WT_3h_VH-up','WT_3h_WM-up',
#                                           'WT_24h_DH-up','WT_24h_MG-up','WT_24h_VH-up','WT_24h_WM-up',
#                                           'WT_72h_DH-up','WT_72h_MG-up','WT_72h_VH-up','WT_72h_WM-up',
#                                           'WT_3h_DH-down','WT_3h_MG-down','WT_3h_VH-down','WT_3h_WM-down',
#                                           'WT_24h_DH-down','WT_24h_MG-down','WT_24h_VH-down','WT_24h_WM-down',
#                                           'WT_72h_DH-down','WT_72h_MG-down','WT_72h_VH-down','WT_72h_WM-down'
#                                          ))
df1<-df1[order(df1$time),]
df1<-df1[order(df1$up_down),]
df1$Description = factor(df1$Description,
                         levels = unique(c(unique(df1[df1$up_down=="up",]$Description),rev(unique(df1[df1$up_down=="down",]$Description)))))  #对Description进

In [ ]:
options(repr.plot.width=6, repr.plot.height=8.5,font.size=15)
library(ggplot2)
p=ggplot(df1,aes(time,Description))+geom_point(aes(size=Count,color=p.adjust))+        #基因数目表示SIZE，p.adjust代表颜色深浅
    scale_color_gradientn(colors=brewer.pal(9,"Spectral"),guide = guide_colorbar(reverse = T))+
    theme_bw()+facet_wrap(~up_down,nrow=2,scales = "free")+
    theme(axis.title = element_blank(),axis.text = element_text(size = 13),axis.text.x.bottom = element_text(angle = 90))#+facet_grid(ONTOLOGY~., scales = "free")
p
ggsave(paste0(sd,"WT.VH.time.Marker.fc05.pct025.p001_GO.pqval001.C5.pval.top5.go_updown.png"),width = 6,height = 8.5,dpi = 400)

In [179]:
df2<-df[grep("WM",df$Cluster),]
df1 = df1 %>% group_by(.,Cluster) %>% filter(Count>=5) %>% top_n(.,-5,p.adjust)  #按照Cluster排序，并取每个cluster的前20个
df1<-df2[df2$Description %in% unique(df1$Description),]
#df1$Cluster<-factor(df1$Cluster,levels = c('WT_3h_DH-up','WT_3h_MG-up','WT_3h_VH-up','WT_3h_WM-up',
#                                           'WT_24h_DH-up','WT_24h_MG-up','WT_24h_VH-up','WT_24h_WM-up',
#                                           'WT_72h_DH-up','WT_72h_MG-up','WT_72h_VH-up','WT_72h_WM-up',
#                                           'WT_3h_DH-down','WT_3h_MG-down','WT_3h_VH-down','WT_3h_WM-down',
#                                           'WT_24h_DH-down','WT_24h_MG-down','WT_24h_VH-down','WT_24h_WM-down',
#                                           'WT_72h_DH-down','WT_72h_MG-down','WT_72h_VH-down','WT_72h_WM-down'
#                                          ))
df1<-df1[order(df1$time),]
df1<-df1[order(df1$up_down),]
df1$Description = factor(df1$Description,
                         levels = unique(c(unique(df1[df1$up_down=="up",]$Description),rev(unique(df1[df1$up_down=="down",]$Description)))))  #对Description进

In [ ]:
options(repr.plot.width=6, repr.plot.height=8.5,font.size=15)
library(ggplot2)
p=ggplot(df1,aes(time,Description))+geom_point(aes(size=Count,color=p.adjust))+        #基因数目表示SIZE，p.adjust代表颜色深浅
    scale_color_gradientn(colors=brewer.pal(9,"Spectral"),guide = guide_colorbar(reverse = T))+
    theme_bw()+facet_wrap(~up_down,nrow=2,scales = "free")+
    theme(axis.title = element_blank(),axis.text = element_text(size = 13),axis.text.x.bottom = element_text(angle = 90))#+facet_grid(ONTOLOGY~., scales = "free")
p
ggsave(paste0(sd,"WT.WM.time.Marker.fc05.pct025.p001_GO.pqval001.C5.pval.top5.go_updown.png"),width = 6,height = 8.5,dpi = 400)

### merge DEG

In [72]:
fp<-c("/home/jovyan/zxli_SCI/result/Seurat/reg.CC/WT_replace_v2/DEGs/SCT/HT/H/WT.SCT.H.time_Marker.p001.csv",
     "/home/jovyan/zxli_SCI/result/Seurat/reg.CC/WT_replace_v2/DEGs/SCT/HT/T/WT.SCT.T.time_Marker.p001.csv",
     "/home/jovyan/zxli_SCI/result/Seurat/reg.CC/WT_replace_v2/DEGs/SCT/distance/WT.SCT.0.5mm.time_Marker.p001.csv",
     "/home/jovyan/zxli_SCI/result/Seurat/reg.CC/WT_replace_v2/DEGs/SCT/distance/WT.SCT.1mm.time_Marker.p001.csv",
     "/home/jovyan/zxli_SCI/result/Seurat/reg.CC/WT_replace_v2/DEGs/SCT/domain_pc20.k50.res025/WM/WT.SCT.WM.time_Marker.p001.csv",
     "/home/jovyan/zxli_SCI/result/Seurat/reg.CC/WT_replace_v2/DEGs/SCT/domain_pc20.k50.res025/MG/WT.SCT.MG.time_Marker.p001.csv",
     "/home/jovyan/zxli_SCI/result/Seurat/reg.CC/WT_replace_v2/DEGs/SCT/domain_pc20.k50.res025/DH/WT.SCT.DH.time_Marker.p001.csv",
     "/home/jovyan/zxli_SCI/result/Seurat/reg.CC/WT_replace_v2/DEGs/SCT/domain_pc20.k50.res025/VH/WT.SCT.VH.time_Marker.p001.csv")

In [ ]:
mark<-time_mark
for(i in 1:length(fp)){
    temp<-read.csv(fp[i],header = T)
    rownames(temp)<-temp[,1]
    temp<-temp[,-1]
    mark<-rbind(mark,temp)
}
dim(mark)
unique(mark$group)

In [78]:
write.csv(mark,"WT.SCT.allgroups.p001.DEGs.merge.csv")